# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
html = requests.get(url).content
html[0:600]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Data science - Wikipedia</title>\n<script>document.documentElement.className=document.documentElement.className.replace(/(^|\\s)client-nojs(\\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Data_science","wgTitle":"Data science","wgCurRevisionId":902790670,"wgRevisionId":902790670,"wgArticleId":35458904,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: '

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(html, "lxml")
table = soup.find_all('a', href=True)

links = {}
for link in table:
    if 'title' in link.attrs: 
        links[link['title']] = link['href']
set(links.values())

{'//en.wikipedia.org/wiki/Wikipedia:Contact_us',
 '//shop.wikimedia.org',
 '/w/index.php?title=Data_science&action=edit',
 '/w/index.php?title=Data_science&action=edit&section=1',
 '/w/index.php?title=Data_science&action=edit&section=2',
 '/w/index.php?title=Data_science&action=edit&section=3',
 '/w/index.php?title=Data_science&action=history',
 '/w/index.php?title=Data_science&action=info',
 '/w/index.php?title=Data_science&oldid=902790670',
 '/w/index.php?title=Data_science&printable=yes',
 '/w/index.php?title=Special:CiteThisPage&page=Data_science&id=902790670',
 '/w/index.php?title=Special:CreateAccount&returnto=Data+science',
 '/w/index.php?title=Special:UserLogin&returnto=Data+science',
 '/wiki/Academic_journal',
 '/wiki/Academic_publishing',
 '/wiki/Academy',
 '/wiki/American_Statistical_Association',
 '/wiki/Analytics',
 '/wiki/Anand_Rajaraman',
 '/wiki/Anomaly_detection',
 '/wiki/Applied_science',
 '/wiki/Artificial_neural_network',
 '/wiki/Artificial_neural_networks',
 '/wiki

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [5]:
domain = 'http://wikipedia.org'

In [6]:
i = 0
j = 0
all_links = list(links.values())

absolute_links = []
relative_links = []

for link in all_links:
    if "%" not in link:
        if link.startswith("/"):
            relative_links.append(domain + link)
        elif link.startswith("http"):
            absolute_links.append(link)
        else:
            print ("Bad Link:", link)

set(relative_links + absolute_links)

{'http://wikipedia.org//en.wikipedia.org/wiki/Wikipedia:Contact_us',
 'http://wikipedia.org//shop.wikimedia.org',
 'http://wikipedia.org/w/index.php?title=Data_science&action=edit',
 'http://wikipedia.org/w/index.php?title=Data_science&action=edit&section=1',
 'http://wikipedia.org/w/index.php?title=Data_science&action=edit&section=2',
 'http://wikipedia.org/w/index.php?title=Data_science&action=edit&section=3',
 'http://wikipedia.org/w/index.php?title=Data_science&action=history',
 'http://wikipedia.org/w/index.php?title=Data_science&action=info',
 'http://wikipedia.org/w/index.php?title=Data_science&oldid=902790670',
 'http://wikipedia.org/w/index.php?title=Data_science&printable=yes',
 'http://wikipedia.org/w/index.php?title=Special:CiteThisPage&page=Data_science&id=902790670',
 'http://wikipedia.org/w/index.php?title=Special:CreateAccount&returnto=Data+science',
 'http://wikipedia.org/w/index.php?title=Special:UserLogin&returnto=Data+science',
 'http://wikipedia.org/wiki/Academic_j

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [7]:
import os

In [8]:
try:
    os.mkdir('wikipedia')
except:
    print ("Directory already exists")
os.listdir('.')

Directory already exists


['main.ipynb', 'wikipedia', '.ipynb_checkpoints']

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [9]:
from slugify import slugify

In [10]:
def index_page(url):
    html = requests.get(url).content
    filename = "./wikipedia/" + slugify(url) + ".html"
    try:
        f= open(filename,'wb')
        f.write(html)
        f.close()
    except:
        pass
    return filename

index_page('https://en.wikipedia.org/wiki/Data_science')

'./wikipedia/https-en-wikipedia-org-wiki-data-science.html'

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [11]:
%%time
for link in set(relative_links + absolute_links):
    index_page(link)
os.listdir('./wikipedia')

CPU times: user 21.4 s, sys: 1.24 s, total: 22.6 s
Wall time: 2min 8s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [12]:
import multiprocessing

In [13]:
# Clean the directory
for link in set(relative_links + absolute_links):
    filename = "./wikipedia/" + slugify(url) + ".html"
    try:
        os.remove(filename)
    except:
        pass

In [14]:
%%time
pool = multiprocessing.Pool()
result = pool.map(index_page, set(relative_links + absolute_links))
pool.terminate()
pool.join()

CPU times: user 46.3 ms, sys: 27.9 ms, total: 74.2 ms
Wall time: 34.3 s
